### Youtube Sentence With Word And Order With Timestamp

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
word_num = 300
df_word = pd.read_csv("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/Turkish/Word Tokenize Merge/Youtube_Word_Tokenize_Merge.csv") # word merge dataframe
df_word = df_word.head(word_num)
df_word

In [ ]:
data_kind = "sentence"
df_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/Turkish/Sentence Clean Merge/Clean_Youtube_{data_kind.lower().capitalize()}_Merge_Result.csv")
df_sent

In [ ]:
word_list = df_word.iloc[:,0].values.tolist()

In [ ]:
sent_list  = df_sent.iloc[:,2].values.tolist()

In [ ]:
resultlist = []

In [ ]:
#import os
#from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
resultlist = manager.list()

def word_in_wordgroup(sent_list):
    mergelist = []
    try:
        word = sent_list.split()
    except:
        pass
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist.append(sent_list)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, sent_list) # string_word liste

In [ ]:
result_list = list(resultlist)
df_result = pd.DataFrame(result_list)
df_result.drop_duplicates(inplace=True)
df_result = df_result.rename(columns = {0: f"{data_kind.lower()}"})
df_sent_merge = pd.merge(df_result, df_sent, how="inner", on=f"{data_kind.lower()}")
df_sent_merge.drop_duplicates(inplace=True)
df_sent_with_words = df_sent_merge.sort_values(by=["video_id","start_time"], ascending=True)
df_sent_with_words.reset_index(drop=True, inplace=True)
df_sent_with_words

In [ ]:
#df_merge_result.to_csv(f"Youtube_{data_kind.capitalize()}_With_{word_num}_Words.csv", index=False)

In [ ]:
df_sent_with_words["with_word"] = 1
df_sent_with_words

In [ ]:
df_with_word = pd.merge(df_sent, df_sent_with_words, how="left", on=["sentence","start_time","end_time","video_id"])
df_with_word.fillna(0, inplace=True)
df_with_word

In [ ]:
df_with_word.with_word.value_counts()

In [ ]:
# First Code Running For All Sentence
zeros = []
ones = []
zero_list = []
one_list = []
for i in range(len(df_with_word)):    
    if df_with_word.loc[i,"with_word"] == 0:
        zeros.append(i)
        try:
            one_list.append((ones[0],len(ones)))
        except:
            pass
        ones = []
    else:
        ones.append(i)
        try:
            zero_list.append((zeros[0],len(zeros)))
        except:
            pass
        zeros = []
try:
    one_list.append((ones[0],len(ones)))
except:
    pass
try:
    zero_list.append((zeros[0],len(zeros)))
except:
    pass

In [ ]:
# Try Code (will test)
zeros = []
ones = []
zero_list = []
one_list = []
for i in range(len(df_with_word)):
    videoid = df_with_word.loc[i,"video_id"]    
    if df_with_word.loc[i,"with_word"] == 0:
        zeros.append(i)
        try:
            one_list.append((ones[0],len(ones)))
        except:
            pass
        ones = []
    else:
        ones.append(i)
        try:
            zero_list.append((zeros[0],len(zeros)))
        except:
            pass
        zeros = []

    if df_with_word.loc[i+1,"video_id"] != videoid:
        try:
            one_list.append((ones[0],len(ones)))
            ones = []
        except:
            pass
        try:
            zero_list.append((zeros[0],len(zeros)))
            zeros = []
        except:
            pass
    else:
        pass

try:
    one_list.append((ones[0],len(ones)))
except:
    pass
try:
    zero_list.append((zeros[0],len(zeros)))
except:
    pass

In [ ]:
len(one_list)

In [ ]:
len(zero_list)

In [ ]:
df_one = pd.DataFrame(one_list)
df_one.rename(columns={0:"index",1:"length"}, inplace=True)
df_one.drop_duplicates(inplace=True)
df_one

In [ ]:
df_one[1].sum()

In [ ]:
df_one.sort_values(by="length", ascending=False)

In [ ]:
df_zero = pd.DataFrame(zero_list)
df_zero.rename(columns={0:"index",1:"length"}, inplace=True)
df_zero.drop_duplicates(inplace=True)
df_zero

In [ ]:
df_zero[1].sum()

In [ ]:
df_word_result = pd.DataFrame()
for i in word_list:
    word_in_video = df_sent_with_words[df_sent_with_words.sentence.str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] # try and except will try for sample 
    word_in_video.insert(0,"word",i)
    df_word_result = pd.concat([df_word_result,word_in_video], axis=0)
df_word_result.reset_index(drop=True, inplace=True)
df_word_result = df_word_result.sort_values(by=["video_id","start_time"], ascending=True)
df_word_result.reset_index(drop=True, inplace=True)
df_word_result

In [ ]:
#df_word_result.to_csv(f"Youtube_{data_kind.capitalize()}_With_{word_num}_Word_And_Words.csv", index=False)

In [ ]:
df_group = pd.DataFrame(df_word_result.groupby("video_id")["word"].nunique())
df_group.reset_index(inplace=True)
df_group.rename(columns={"word":"frequency"}, inplace=True)
df_group.sort_values(by="frequency", ascending=False, inplace=True)
df_group.reset_index(drop=True, inplace=True)
df_group 

In [ ]:
df_word_result[df_word_result.video_id == "UDDfBEUGY60"]